In [15]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from nltk.corpus import stopwords
import pickle
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [2]:
df = pd.read_csv("rows.csv")

/var/folders/k7/w37qytjn78j3258njhbzvlvr0000gn/T/ipykernel_96059/627803143.py:1: DtypeWarning: Columns (4,5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("rows.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282355 entries, 0 to 1282354
Data columns (total 18 columns):
 #   Column                        Non-Null Count    Dtype 
---  ------                        --------------    ----- 
 0   Date received                 1282355 non-null  object
 1   Product                       1282355 non-null  object
 2   Sub-product                   1047189 non-null  object
 3   Issue                         1282355 non-null  object
 4   Sub-issue                     751169 non-null   object
 5   Consumer complaint narrative  383564 non-null   object
 6   Company public response       449082 non-null   object
 7   Company                       1282355 non-null  object
 8   State                         1262955 non-null  object
 9   ZIP code                      1167057 non-null  object
 10  Tags                          175643 non-null   object
 11  Consumer consent provided?    690654 non-null   object
 12  Submitted via                 1282355 non-

In [7]:
df = df[["Date received", "Product", "Sub-product", "Issue", "Sub-issue", "Consumer complaint narrative"]]

In [8]:
df.shape

(1282355, 6)

In [9]:
df.isnull().sum()

Date received                        0
Product                              0
Sub-product                     235166
Issue                                0
Sub-issue                       531186
Consumer complaint narrative    898791
dtype: int64

In [10]:
df = df.dropna(how="any")

In [11]:
df.shape

(238303, 6)

In [12]:
df["Product"].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    89812
Debt collection                                                                 86710
Student loan                                                                    21806
Credit card or prepaid card                                                     21102
Checking or savings account                                                     12870
Vehicle loan or lease                                                            5742
Payday loan, title loan, or personal loan                                         261
Name: Product, dtype: int64

In [16]:
types = df["Product"].value_counts()

values = types.values
names = types.index

fig = px.pie(df, values=values, names=names, hole=0.5)
fig.show()

In [19]:
stop_words_list = stopwords.words("english")

In [20]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\d", " ", text)
    text = re.sub(r"<.*?>+", "", text)
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("https?://S+|www.\.\S+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    words = text.split()
    words = [word for word in words if not word in stop_words_list]
    words = [re.sub(r"(.)\1{1,}", r"\1\1", word) for word in words]
    words = [word.strip() for word in words if len(word.strip()) > 1]
    text = " ".join(words)
    return text

In [21]:
df["Consumer complaint narrative"] = df["Consumer complaint narrative"].apply(preprocess_text)

In [22]:
X = df["Consumer complaint narrative"]
X = np.array(X)

y = df["Product"]
y = np.array(y)

In [23]:
cv = CountVectorizer()
X_scaled = cv.fit_transform(X)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=4242)

In [25]:
model = SGDClassifier()
model.fit(X_train, y_train)

SGDClassifier()

In [26]:
model.score(X_test, y_test)

0.8481347302457652

In [28]:
pickle.dump(model, open("model.pkl", "wb"))

In [29]:
pickle.dump(cv, open("cv.pkl", "wb"))